#***1 Классификация ***

База про спортсменов



##Креативная часть: найти парамеры и комбинации их, которые будут увеличивать accuracy


In [1]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

In [2]:
##Для отображения плотов на лету
%matplotlib inline

In [49]:
f='athletes_sochi.txt'
dfs = pd.read_csv(f).dropna()

In [50]:
dfs = pd.read_csv(f).dropna()

In [51]:
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


In [52]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)

In [53]:
dfs.head()

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country,BMI
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States,22.985398
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy,24.835646
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan,21.461937
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand,23.702162
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland,18.612245


Подготовим входные данные

набор пар для входных векторов X: пары это вес и рост
набор пар для выходных векторов Y: это значения 1 -- male, 0 -- female
перемешать набор данных весь и взять 20% отложить как test, остальное будет train

In [54]:
alldata = dfs[ ['weight', 'height', 'BMI', 'gender'] ]
alldata

,weight,height,BMI,gender
0,68.0,1.72,22.985398,Male
1,85.0,1.85,24.835646,Male
2,68.0,1.78,21.461937,Male
4,82.0,1.86,23.702162,Male
5,57.0,1.75,18.612245,Male
...,...,...,...,...
2851,98.0,1.93,26.309431,Male
2853,60.0,1.70,20.761246,Female
2854,65.0,1.68,23.030045,Female
2856,68.0,1.76,21.952479,Male


In [57]:
alldata = alldata.sample(frac=1)

In [58]:
alldata

,weight,height,BMI,gender
2555,91.0,1.82,27.472527,Male
2353,71.0,1.65,26.078972,Male
2231,84.0,1.77,26.812219,Male
465,68.0,1.70,23.529412,Female
429,62.0,1.71,21.203105,Female
...,...,...,...,...
1245,91.0,1.83,27.173102,Male
1506,56.0,1.67,20.079601,Female
1568,94.0,1.88,26.595745,Male
669,75.0,1.81,22.893074,Male


In [59]:
allinput = alldata[ ['weight', 'height', 'BMI'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))

In [60]:
allinput

,weight,height,BMI
2555,91.0,1.82,27.472527
2353,71.0,1.65,26.078972
2231,84.0,1.77,26.812219
465,68.0,1.70,23.529412
429,62.0,1.71,21.203105
...,...,...,...
1245,91.0,1.83,27.173102
1506,56.0,1.67,20.079601
1568,94.0,1.88,26.595745
669,75.0,1.81,22.893074


In [61]:
alloutput

2555    1
2353    1
2231    1
465     0
429     0
       ..
1245    1
1506    0
1568    1
669     1
2801    1
Name: gender, Length: 2479, dtype: int64

In [62]:
chunk_80 = int(len(alldata) * 0.8)

X_train = allinput[ :chunk_80]
Y_train = alloutput[ :chunk_80]

X_test = allinput[chunk_80: ]
Y_test = alloutput[chunk_80: ]

In [ ]:
#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)

In [63]:
model = linear_model.LogisticRegression()

In [64]:
model.fit(X_train, Y_train)

LogisticRegression()

In [65]:
model.coef_

array([[ 0.37030571,  0.19884459, -0.80332624]])

In [66]:
Y_test_predicted = model.predict(X_test)

In [67]:
Y_test_predicted

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,

In [68]:
[tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
confusion_matrix(Y_test, Y_test_predicted)

array([[171,  35],
       [ 35, 255]])

In [69]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  (precision * recall) / ((precision) + recall))

Precision:  0.8793103448275862
Recall:  0.8793103448275862
Accuracy:  0.8588709677419355
F1:  0.8583450210378681


In [70]:
from sklearn.metrics import classification_report

In [71]:
report = classification_report(Y_test, model.predict(X_test), target_names=['Male', 'Female'])
print(report)

              precision    recall  f1-score   support

        Male       0.83      0.83      0.83       206
      Female       0.88      0.88      0.88       290

    accuracy                           0.86       496
   macro avg       0.85      0.85      0.85       496
weighted avg       0.86      0.86      0.86       496



In [72]:
women = alldata.loc[alldata.gender == 'Female']

In [73]:
men = alldata.loc[alldata.gender == 'Male']

In [74]:
women['height'].mean()

1.6765010351966874

In [75]:
women['height'].max()

1.86

In [76]:
women['weight'].mean()

61.69979296066253

In [77]:
women['weight'].max()

92.0

In [78]:
men['height'].mean()

1.8090218109715797

In [79]:
men['height'].min()

1.57

In [80]:
men['weight'].mean()

80.39061467283543

In [81]:
men['weight'].min()

47.0

In [88]:
q1 = women['height'].quantile(0.25)
q3 = women['height'].quantile(0.75)
iqr = q3 - q1
df1 = women[(women['height'] < q3 + iqr)]

In [89]:
df1['height'].max()

1.8

In [90]:
q1 = women['weight'].quantile(0.25)
q3 = women['weight'].quantile(0.75)
iqr = q3 - q1
df1 = women[(women['weight'] < q3 + iqr)]

In [91]:
df1['weight'].max()

75.0

In [103]:
q1 = men['height'].quantile(0.25)
q3 = men['height'].quantile(0.75)
iqr = q3 - q1
df2 = men[(men['height'] > q3 - iqr)]

In [104]:
df2['height'].min()

1.78

In [106]:
q1 = men['weight'].quantile(0.25)
q3 = men['weight'].quantile(0.75)
iqr = q3 - q1
df2 = men[(men['weight'] > q3 - iqr)]

In [107]:
df2['weight'].min()

73.0

In [108]:
df = pd.concat([df1, df2], axis=0)

In [109]:
df

,weight,height,BMI,gender
465,68.0,1.70,23.529412,Female
429,62.0,1.71,21.203105,Female
1309,60.0,1.63,22.582709,Female
1013,63.0,1.73,21.049818,Female
1357,65.0,1.63,24.464602,Female
...,...,...,...,...
1586,76.0,1.80,23.456790,Male
1219,86.0,1.80,26.543210,Male
1245,91.0,1.83,27.173102,Male
1568,94.0,1.88,26.595745,Male


In [112]:
df = df.sample(frac=1)

In [113]:
df

,weight,height,BMI,gender
1424,61.0,1.73,20.381570,Female
1029,95.0,1.88,26.878678,Male
1210,97.0,1.89,27.154895,Male
2721,79.0,1.85,23.082542,Male
1160,95.0,1.93,25.504040,Male
...,...,...,...,...
2700,53.0,1.65,19.467401,Female
67,68.0,1.69,23.808690,Female
2221,100.0,1.93,26.846358,Male
2051,91.0,1.87,26.023049,Male


In [114]:
dfinput = df[ ['weight', 'height', 'BMI'] ]
dfoutput = df["gender"].apply(lambda x: int(x == "Male"))

In [115]:
dfinput

,weight,height,BMI
1424,61.0,1.73,20.381570
1029,95.0,1.88,26.878678
1210,97.0,1.89,27.154895
2721,79.0,1.85,23.082542
1160,95.0,1.93,25.504040
...,...,...,...
2700,53.0,1.65,19.467401
67,68.0,1.69,23.808690
2221,100.0,1.93,26.846358
2051,91.0,1.87,26.023049


In [116]:
dfoutput

1424    0
1029    1
1210    1
2721    1
1160    1
       ..
2700    0
67      0
2221    1
2051    1
2668    0
Name: gender, Length: 2010, dtype: int64

In [117]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(dfinput, dfoutput, test_size=0.2)

In [118]:
model_ = linear_model.LogisticRegression()

In [119]:
model_.fit(X_train, Y_train)

LogisticRegression()

In [120]:
model_.coef_

array([[ 1.58949969,  0.05601171, -1.02073107]])

In [121]:
Y_test_predicted = model_.predict(X_test)

In [122]:
Y_test_predicted

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [123]:
[tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
confusion_matrix(Y_test, Y_test_predicted)

array([[177,   5],
       [  5, 215]])

In [124]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  (precision * recall) / ((precision) + recall))

Precision:  0.9772727272727273
Recall:  0.9772727272727273
Accuracy:  0.9751243781094527
F1:  0.8403908794788273


In [125]:
from sklearn.metrics import classification_report

In [126]:
report = classification_report(Y_test, model.predict(X_test), target_names=['Male', 'Female'])
print(report)

              precision    recall  f1-score   support

        Male       0.99      0.88      0.93       182
      Female       0.91      1.00      0.95       220

    accuracy                           0.94       402
   macro avg       0.95      0.94      0.94       402
weighted avg       0.95      0.94      0.94       402



##Техническая часть: сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [127]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

def calculate_classification_metrics(X_test, y_test, model):

    y_pred = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    return {
        "Precision": precision,
        "Recall": recall,
        "Accuracy": accuracy
    }

In [128]:
metrics = calculate_classification_metrics(X_test, Y_test, model_)
metrics

{'Precision': 0.9772727272727273,
 'Recall': 0.9772727272727273,
 'Accuracy': 0.9751243781094527}

#**2 Кластеризация**

## Креативная часть: Релизовать на питоне ДЗ от вашего преподователя по МО


Если это про матан то сделано


##Техническая часть: сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [ ]:
Реализовано по матану